# Code description

In this notebook, we verify the part of the proof of Lemma C.8 from 

> "Convergence of Proximal Point and Extragradient-Based Methods Beyond Monotonicity: the Case of Negative Comonotonicity"

that involves estimation of $T_k$.

## Problem

We consider the following problem
       $$ \text{find } x^\ast \text{ such that }F(x^\ast) = 0$$
where $F:\mathbb{R}^d \to \mathbb{R}^d$ is $\rho$-negative comonotone and $L$-Lipschitz operator.

## Algorithm

We consider Optimistic Gradient method (OG) with same-stepsize policy: $\tilde{x}^0 = x^0$ and for all $k > 0$
$$
\tilde{x}^k = x^k - \gamma F(\tilde{x}^{k-1}),
$$
$$
x^{k+1} = x^k - \gamma F(\tilde{x}^k)
$$

## The goal

Let $k \geq 1$ and 

$$
T_k = \frac{2\rho}{\gamma}\|F(x^{k}) - F(x^{k+1})\|^2 + \left(\frac{1}{3} + 3L^2\gamma^2\right)\|F(\tilde{x}^{k-1}) - F(\tilde{x}^{k})\|^2 - \frac{2}{3}\left(1 - \frac{\rho}{\gamma}\right)\|F(x^{k+1}) - F(\tilde{x}^k)\|^2
$$
$$
 - \|F(x^k) - F(\tilde{x}^{k-1})\|^2 - \frac{1}{3}\|F(x^{k+1}) - F(\tilde{x}^{k-1})\|^2 - \|F(x^{k}) - F(\tilde{x}^k)\|^2.
$$

In the proof of Lemma C.8, we derive
$$
\Psi_{k+1} - \Psi_k \leq T_k,
$$
where $\Psi_k = \|F(x^{k})\|^2 + \|F(x^{k}) - F(\tilde{x}^{k-1})\|^2$.

Below using the symbolic computations we verify the formula for $T_k$ and also show numerically that
$$
\begin{pmatrix} -\frac{1}{3} & -\frac{1}{2} &  \frac{1}{2} & \frac{1}{3} \\ -\frac{1}{2} & -\frac{3}{2} & 1 & 1\\ \frac{1}{2} & 1 & -\frac{4927}{5766} & - \frac{1861}{2883}\\ \frac{1}{3} & 1 & - \frac{1861}{2883} & - \frac{661}{961}\end{pmatrix} \preccurlyeq -\frac{1}{100}\begin{pmatrix} 0 & 0 &  0 & 0 \\ 0 & 0 &  0 & 0\\ 0 & 0 &  1 & -1\\ 0 & 0 &  -1 & 1\end{pmatrix}.
$$

In [1]:
import sympy
from sympy import Matrix, symbols, Rational

In [2]:
L = symbols('L', positive=True)
gamma = symbols('g', positive=True)
rho = symbols('r', positive=True)

xk2 = symbols('x_k+1') #this is $x^{k+1}$
xk1 = symbols('x_k') #this is $x^k$

xtk1 = symbols('y_k+1') #this is $\tilde{x}^{k}$
xtk = symbols('y_k') #this is $\tilde{x}^{k-1}$

In [3]:
#Left-hand sides of inequalities that we sum up in the proof of Lemma C.4
LHS_1 = - 2/3 * rho / gamma * (xk2 - xtk1)**2
LHS_2 = - 2 * rho / gamma * (xk1 - xk2)**2
LHS_3 = 3 * (xk2 - xtk1)**2

#Right-hand sides of inequalities that we sum up in the proof of Lemma C.4
RHS_1 = 2/3 * (xk2 - xtk1) * (xtk - xtk1)
RHS_2 = 2 * (xk1 - xk2) * (xtk1)
RHS_3 = 3 * L**2 * gamma**2 * (xtk - xtk1)**2

#Potentials
Psik1 = (xk2)**2 + (xk2 - xtk1)**2
Psik = (xk1)**2 + (xk1 - xtk)**2

In [4]:
# Below we compute $T_k$ using symbolic computations
Tk_derived_by_sympy = (RHS_1 + RHS_2 + RHS_3) - (LHS_1 + LHS_2 + LHS_3) + Psik1 - Psik

In [5]:
# Expression we derive in Lemma C.4
Term1 = 2*rho/gamma * (xk1 - xk2)**2
Term2 = (1/3 + 3 * L**2 * gamma**2) * (xtk - xtk1)**2
Term3 = - 2/3 * (1 - rho/gamma) * (xk2 - xtk1)**2
Term4 = - (xk1 - xtk)**2
Term5 = - 1/3 * (xk2 - xtk)**2
Term6 = - (xk1 - xtk1)**2

Tk_derived_in_Lemma_C4 = Term1 + Term2 + Term3 + Term4 + Term5 + Term6

In [6]:
Tk_derived_by_sympy.expand().simplify(rational=True)

(g*(9*L**2*g**2*y_k**2 - 18*L**2*g**2*y_k*y_k+1 + 9*L**2*g**2*y_k+1**2 - 6*x_k**2 + 6*x_k*y_k + 6*x_k*y_k+1 - 3*x_k+1**2 + 2*x_k+1*y_k + 4*x_k+1*y_k+1 - 3*y_k**2 - 2*y_k*y_k+1 - 4*y_k+1**2) + 6*r*x_k**2 - 12*r*x_k*x_k+1 + 8*r*x_k+1**2 - 4*r*x_k+1*y_k+1 + 2*r*y_k+1**2)/(3*g)

In [7]:
Tk_derived_in_Lemma_C4.expand().simplify(rational=True)

(g*(9*L**2*g**2*y_k**2 - 18*L**2*g**2*y_k*y_k+1 + 9*L**2*g**2*y_k+1**2 - 6*x_k**2 + 6*x_k*y_k + 6*x_k*y_k+1 - 3*x_k+1**2 + 2*x_k+1*y_k + 4*x_k+1*y_k+1 - 3*y_k**2 - 2*y_k*y_k+1 - 4*y_k+1**2) + 6*r*x_k**2 - 12*r*x_k*x_k+1 + 8*r*x_k+1**2 - 4*r*x_k+1*y_k+1 + 2*r*y_k+1**2)/(3*g)

In [8]:
difference = Tk_derived_in_Lemma_C4 - Tk_derived_by_sympy
difference.expand().simplify()

x_k+1*(1.11022302462516e-16*x_k+1 - 3.33066907387547e-16*y_k+1)

The above three cells verify the correctness of our derivations of inequality $\Phi_{k+1} - \Phi_k \leq T_k$

### Matrix form of $T_k$

One can rewrite $T_k$ as
$$
T_k = \begin{pmatrix} F(x^{k+1}) \\ F(x^k) \\  F(\tilde{x}^{k}) \\  F(\tilde{x}^{k-1})\end{pmatrix}^\top \left(A(\gamma, L, \rho) \otimes  I_d\right) \begin{pmatrix} F(x^{k+1}) \\ F(x^k) \\  F(\tilde{x}^{k}) \\  F(\tilde{x}^{k-1})\end{pmatrix},
$$
where $A(\gamma, L, \rho) \in \mathbb{R}^{d\times d}$ is some $d\times d$ matrix that can be derived explicitly.
In the proof, we use that $A(\gamma, L, \rho) \preccurlyeq A\left(\frac{10}{31 L}, L, \frac{5}{62L}\right)$. Below we compute $A(\gamma, L, \rho)$ and $A\left(\frac{10}{31 L}, L, \frac{5}{62L}\right)$ symbolically. 

In [9]:
matrix_A = Matrix([
    [ -gamma + Rational(8,3) * rho, -2*rho, Rational(2,3) * gamma - Rational(2,3) * rho, Rational(1,3) * gamma],
    [ -2*rho, -2 * gamma + 2 * rho, gamma, gamma],
    [ Rational(2,3) * gamma - Rational(2,3) * rho, gamma,  3 * (L**2) * (gamma**3) - Rational(4,3) * gamma + Rational(2,3) * rho, -3 * L**2 * gamma**3 - Rational(1,3) * gamma],
    [ Rational(1,3) * gamma, gamma, -3 * L**2 * gamma**3 - Rational(1,3) * gamma, 3 * L**2 * gamma**3 - gamma]]) / gamma  # xk2, xk1, yk1, yk

In [10]:
matrix_A.expand()

Matrix([
[ -1 + 8*r/(3*g),     -2*r/g,               2/3 - 2*r/(3*g),                1/3],
[         -2*r/g, -2 + 2*r/g,                             1,                  1],
[2/3 - 2*r/(3*g),          1, 3*L**2*g**2 - 4/3 + 2*r/(3*g), -3*L**2*g**2 - 1/3],
[            1/3,          1,            -3*L**2*g**2 - 1/3,    3*L**2*g**2 - 1]])

In [11]:
matrix_A.subs({gamma: Rational(10,31), L : Rational(1,1), rho : Rational(5,62)})

Matrix([
[-1/3, -1/2,        1/2,        1/3],
[-1/2, -3/2,          1,          1],
[ 1/2,    1, -4927/5766, -1861/2883],
[ 1/3,    1, -1861/2883,   -661/961]])

That is, the code above verifies that for $\gamma \leq \frac{10}{31L}$ and $\rho \leq \frac{5}{62L}$
$$
T_k \leq \begin{pmatrix} F(x^{k+1}) \\ F(x^k) \\  F(\tilde{x}^{k}) \\  F(\tilde{x}^{k-1})\end{pmatrix}^\top \left(\begin{pmatrix} -\frac{1}{3} & -\frac{1}{2} &  \frac{1}{2} & \frac{1}{3} \\ -\frac{1}{2} & -\frac{3}{2} & 1 & 1\\ \frac{1}{2} & 1 & -\frac{4927}{5766} & - \frac{1861}{2883}\\ \frac{1}{3} & 1 & - \frac{1861}{2883} & - \frac{661}{961}\end{pmatrix} \otimes  I_d\right) \begin{pmatrix} F(x^{k+1}) \\ F(x^k) \\  F(\tilde{x}^{k}) \\  F(\tilde{x}^{k-1})\end{pmatrix}
$$

### Estimation of $A\left(\frac{10}{31 L}, L, \frac{5}{62L}\right)$

In [12]:
import numpy as np

In [13]:
L = 1.0
gamma = 1.0/(3.1*L)
rho = gamma/4

matrix_A_numerical = np.array([[ -gamma + 8/3 * rho, -2*rho, 2/3 * gamma - 2/3 * rho, 1/3 * gamma],
    [ -2*rho, -2 * gamma + 2 * rho, gamma, gamma],
    [ 2/3 * gamma - 2/3 * rho, gamma,  3 * (L**2) * (gamma**3) - 4/3 * gamma + 2/3 * rho, -3 * L**2 * gamma**3 - 1/3 * gamma],
    [ 1/3 * gamma, gamma, -3 * L**2 * gamma**3 - 1/3 * gamma, 3 * L**2 * gamma**3 - gamma]]) / gamma
print(matrix_A_numerical)

[[-0.33333333 -0.5         0.5         0.33333333]
 [-0.5        -1.5         1.          1.        ]
 [ 0.5         1.         -0.85449185 -0.64550815]
 [ 0.33333333  1.         -0.64550815 -0.68782518]]


In [14]:
# this matrix is negative semidefinite
np.linalg.eigvals(matrix_A_numerical)

array([-3.09070413e+00, -2.58459031e-01, -2.64871998e-02, -1.01919124e-16])

In [15]:
alpha = -0.01
upper_bound_matrix = np.array([[0., 0., 0., 0.],
                       [0., 0., 0., 0.],
                       [0., 0., 1., -1.],
                       [0., 0., -1., 1.]])

print("Eigenvalues: ", np.linalg.eigvals(alpha*upper_bound_matrix - matrix_A_numerical))
print("Eigenvalues are positive")
print("Determinant: ", np.linalg.det(alpha*upper_bound_matrix - matrix_A_numerical))
print("Determinant is non-negative")

Eigenvalues:  [3.09068659e+00 2.50554248e-01 6.17315407e-17 1.44095255e-02]
Eigenvalues are positive
Determinant:  6.179481442698483e-19
Determinant is non-negative


This implies that
$$
\begin{pmatrix} -\frac{1}{3} & -\frac{1}{2} &  \frac{1}{2} & \frac{1}{3} \\ -\frac{1}{2} & -\frac{3}{2} & 1 & 1\\ \frac{1}{2} & 1 & -\frac{4927}{5766} & - \frac{1861}{2883}\\ \frac{1}{3} & 1 & - \frac{1861}{2883} & - \frac{661}{961}\end{pmatrix} \preccurlyeq -\frac{1}{100}\begin{pmatrix} 0 & 0 &  0 & 0 \\ 0 & 0 &  0 & 0\\ 0 & 0 &  1 & -1\\ 0 & 0 &  -1 & 1\end{pmatrix}
$$